In [10]:
import numpy as np
import pandas as pd
import cv2
import os

In [11]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [12]:
print(os.listdir("/gdrive/My Drive/DerinOgrenmeProje/lung_image_sets"))

['lung_squamous cell carcinoma', 'lung_adenocarcinoma', 'lung_benign tissue']


In [13]:
datadir = '/gdrive/My Drive/DerinOgrenmeProje/lung_image_sets'
categories = ['lung_benign tissue', 'lung_squamous cell carcinoma', 'lung_adenocarcinoma']
all_data = []
img_size = 128

In [14]:
def create_all_data():
    for category in categories:
        path=str(os.path.join(datadir, category))
        class_num=categories.index(category)
        count = 0
        for img in os.listdir(path):
            img_array=cv2.imread(os.path.join(path,img))
            new_array=cv2.resize(img_array,(img_size,img_size), interpolation=cv2.INTER_AREA)
            all_data.append([new_array,class_num])
            count += 1
        print(class_num,path,count)

create_all_data()

0 /gdrive/My Drive/DerinOgrenmeProje/lung_image_sets/lung_benign tissue 5000
1 /gdrive/My Drive/DerinOgrenmeProje/lung_image_sets/lung_squamous cell carcinoma 5000
2 /gdrive/My Drive/DerinOgrenmeProje/lung_image_sets/lung_adenocarcinoma 5000


In [15]:
print("Total Image: ", len(all_data))

Total Image:  15000


In [16]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [18]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [19]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(8775, 128, 128, 3)
(5250, 128, 128, 3)
(975, 128, 128, 3)
(8775,)
(5250,)
(975,)


In [20]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=3)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=3)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=3)

In [21]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

def create_model(learn_rate):
    model = Sequential()

    model.add(Conv2D(32, (4,4), padding='same', activation='relu',input_shape=(img_size, img_size, 3)))
    model.add(Conv2D(32, (4,4), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(64, (4,4), padding='same', activation='relu'))
    model.add(Conv2D(64, (4,4), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (4,4), padding='same', activation='relu'))
    model.add(Conv2D(128, (4,4), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))

# epoch başı 195-293 iterasyon
    learn_rate = ExponentialDecay(learn_rate, decay_steps=1300, decay_rate=0.96, staircase=True)
    optimizer=Adam(learning_rate=learn_rate,beta_1=0.9,beta_2=0.999)

    model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])
    return model

In [22]:
!pip install scikeras

In [23]:
from scikeras.wrappers import KerasClassifier

param_grid = {
    'learn_rate': [0.005, 0.01, 0.05]
}

model = KerasClassifier(build_fn=create_model,batch_size=32, epochs=5, learn_rate=0.005)

In [24]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid.fit(x_train, train_yCl)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
183/183 [==============================] - 40s 105ms/step - loss: 5.9269 - accuracy: 0.6795
Epoch 2/5
183/183 [==============================] - 16s 88ms/step - loss: 0.8841 - accuracy: 0.7431
Epoch 3/5
183/183 [==============================] - 16s 90ms/step - loss: 0.4654 - accuracy: 0.8265
Epoch 4/5
183/183 [==============================] - 17s 92ms/step - loss: 0.3755 - accuracy: 0.8386
Epoch 5/5
92/92 [==============================] - 3s 27ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
183/183 [==============================] - 20s 93ms/step - loss: 3.9441 - accuracy: 0.7817
Epoch 2/5
183/183 [==============================] - 17s 92ms/step - loss: 0.3575 - accuracy: 0.8602
Epoch 3/5
183/183 [==============================] - 17s 92ms/step - loss: 0.3153 - accuracy: 0.8805
Epoch 4/5
183/183 [==============================] - 17s 92ms/step - loss: 0.2882 - accuracy: 0.8868
Epoch 5/5
92/92 [==============================] - 2s 22ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/5
183/183 [==============================] - 20s 89ms/step - loss: 58.7878 - accuracy: 0.3431
Epoch 2/5
183/183 [==============================] - 16s 88ms/step - loss: 1.1014 - accuracy: 0.3395
Epoch 3/5
183/183 [==============================] - 16s 88ms/step - loss: 1.1035 - accuracy: 0.3412
Epoch 4/5
183/183 [==============================] - 16s 88ms/step - loss: 1.1012 - accuracy: 0.3388
Epoch 5/5
92/92 [==============================] - 2s 21ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
183/183 [==============================] - 20s 90ms/step - loss: 184.4776 - accuracy: 0.4352
Epoch 2/5
183/183 [==============================] - 16s 90ms/step - loss: 1.2292 - accuracy: 0.3791
Epoch 3/5
183/183 [==============================] - 16s 89ms/step - loss: 1.5091 - accuracy: 0.3800
Epoch 4/5
183/183 [==============================] - 16s 88ms/step - loss: 1.1327 - accuracy: 0.3369
Epoch 5/5
92/92 [==============================] - 2s 21ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
275/275 [==============================] - 31s 99ms/step - loss: 8.1681 - accuracy: 0.6239
Epoch 2/5
275/275 [==============================] - 25s 92ms/step - loss: 0.7641 - accuracy: 0.6150
Epoch 3/5
275/275 [==============================] - 25s 90ms/step - loss: 0.7695 - accuracy: 0.5995
Epoch 4/5
275/275 [==============================] - 24s 89ms/step - loss: 0.6585 - accuracy: 0.6122
Epoch 5/5
275/275 [==============================] - 25s 90ms/step - loss: 0.9028 - accuracy: 0.5928


In [25]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.624160 using {'learn_rate': 0.01}


In [26]:
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'learn_rate': uniform(0.005, 0.1)
}

random = RandomizedSearchCV(estimator=model, param_distributions=param_dist, scoring='accuracy', cv=3, n_iter=3)
random_result = random.fit(x_train, train_yCl)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
183/183 [==============================] - 27s 96ms/step - loss: 90.8243 - accuracy: 0.4234
Epoch 2/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1065 - accuracy: 0.3316
Epoch 3/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1009 - accuracy: 0.3350
Epoch 4/5
183/183 [==============================] - 16s 88ms/step - loss: 1.1014 - accuracy: 0.3138
Epoch 5/5
92/92 [==============================] - 2s 21ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
183/183 [==============================] - 21s 90ms/step - loss: 180.1013 - accuracy: 0.5667
Epoch 2/5
183/183 [==============================] - 17s 91ms/step - loss: 4.4814 - accuracy: 0.5021
Epoch 3/5
183/183 [==============================] - 16s 90ms/step - loss: 1.2587 - accuracy: 0.3279
Epoch 4/5
183/183 [==============================] - 16s 88ms/step - loss: 1.2096 - accuracy: 0.3301
Epoch 5/5
92/92 [==============================] - 2s 20ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/5
183/183 [==============================] - 20s 90ms/step - loss: 27.7217 - accuracy: 0.3467
Epoch 2/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1256 - accuracy: 0.3344
Epoch 3/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1021 - accuracy: 0.3316
Epoch 4/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1110 - accuracy: 0.3398
Epoch 5/5
92/92 [==============================] - 2s 21ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
183/183 [==============================] - 20s 90ms/step - loss: 43.8213 - accuracy: 0.3439
Epoch 2/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1010 - accuracy: 0.3484
Epoch 3/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1006 - accuracy: 0.3328
Epoch 4/5
183/183 [==============================] - 16s 89ms/step - loss: 1.1001 - accuracy: 0.3422
Epoch 5/5
92/92 [==============================] - 2s 20ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
275/275 [==============================] - 28s 90ms/step - loss: 272.8087 - accuracy: 0.4250
Epoch 2/5
275/275 [==============================] - 24s 88ms/step - loss: 1.3500 - accuracy: 0.3333
Epoch 3/5
275/275 [==============================] - 24s 88ms/step - loss: 1.1028 - accuracy: 0.3297
Epoch 4/5
275/275 [==============================] - 24s 88ms/step - loss: 1.1038 - accuracy: 0.3319
Epoch 5/5
275/275 [==============================] - 24s 88ms/step - loss: 1.1024 - accuracy: 0.3329


In [27]:
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.344615 using {'learn_rate': 0.08091432741932086}


In [28]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, verbose=3)
]

In [29]:
history = create_model(0.005).fit(x_train, train_yCl,
                    batch_size=32,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 13
                    )

Epoch 1/13
274/275 [============================>.] - ETA: 0s - loss: 3.2787 - accuracy: 0.7681
Epoch 1: val_accuracy improved from -inf to 0.64000, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


275/275 [==============================] - 32s 106ms/step - loss: 3.2767 - accuracy: 0.7681 - val_loss: 1.1591 - val_accuracy: 0.6400
Epoch 2/13
274/275 [============================>.] - ETA: 0s - loss: 0.3717 - accuracy: 0.8472
Epoch 2: val_accuracy improved from 0.64000 to 0.82564, saving model to model.h5
275/275 [==============================] - 28s 101ms/step - loss: 0.3715 - accuracy: 0.8473 - val_loss: 0.4817 - val_accuracy: 0.8256
Epoch 3/13
274/275 [============================>.] - ETA: 0s - loss: 0.2966 - accuracy: 0.8903
Epoch 3: val_accuracy improved from 0.82564 to 0.89333, saving model to model.h5
275/275 [==============================] - 27s 100ms/step - loss: 0.2964 - accuracy: 0.8904 - val_loss: 0.2860 - val_accuracy: 0.8933
Epoch 4/13
274/275 [============================>.] - ETA: 0s - loss: 0.2761 - accuracy: 0.8975
Epoch 4: val_accuracy did not improve from 0.89333
275/275 [==============================] - 27s 96ms/step - loss: 0.2822 - accuracy: 0.8971 - val_

In [30]:
model = keras.models.load_model('/content/model.h5')

In [31]:
score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid)

score_test = model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test)

score_train = model.evaluate(x_train, train_yCl)
print("Train Accuracy: ", score_train)

31/31 [==============================] - 1s 24ms/step - loss: 0.1760 - accuracy: 0.9323
Validation Accuracy:  [0.1759505718946457, 0.9323077201843262]
165/165 [==============================] - 4s 23ms/step - loss: 0.2038 - accuracy: 0.9333
Test Accuracy:  [0.2038230150938034, 0.9333333373069763]
275/275 [==============================] - 6s 23ms/step - loss: 0.1080 - accuracy: 0.9567
Train Accuracy:  [0.10802633315324783, 0.9566951394081116]


In [32]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix,classification_report

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    confusion_mat = confusion_matrix(y_true, y_pred)
    class_report = classification_report(y_true, y_pred)
    return accuracy, f1, precision, recall, confusion_mat, class_report

In [33]:
y_pred = model.predict(x_test)

165/165 [==============================] - 4s 21ms/step


In [34]:
accuracy, f1, precision, recall, confusion_mat, class_report = calculate_metrics(test_yCl.argmax(axis=1),y_pred.argmax(axis=1))

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Skoru: {f1:.4f}\n")

print("\nKarmaşıklık Matrisi:")
print(confusion_mat)

print("\nSınıflandırma Raporu:")
print(class_report)

Test Accuracy: 0.9333
Test Precision: 0.9350
Test Recall: 0.9333
Test F1 Skoru: 0.9336


Karmaşıklık Matrisi:
[[1698    0   80]
 [   0 1679   75]
 [   1  194 1523]]

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1778
           1       0.90      0.96      0.93      1754
           2       0.91      0.89      0.90      1718

    accuracy                           0.93      5250
   macro avg       0.93      0.93      0.93      5250
weighted avg       0.93      0.93      0.93      5250

